In [1]:
from tardis.atomic import AtomData
import numpy as np
import pandas as pd

atomic_dataset = AtomData.from_hdf5('kurucz_cd23_chianti_H_He.h5')

tardis.atomic - INFO - Read Atom Data with UUID=5ca3035ca8b311e3bb684437e69d75d7 and MD5=21095dd25faa1683f4c90c911a00c3f8


In [2]:
def get_index(ele):
    index=-1
    for atomic_no,row in atomic_dataset.atom_data.iterrows():
        if ele in row['name']:
            index = atomic_no
            break
    return index

In [45]:
def extract_block(f):
    qty = []
    for line in f:
        items = line.split()
        if items:
            qty.extend(np.array(items).astype(np.float64))
        else:
            break
    qty = np.array(qty)
    return qty

In [48]:
def parse_file(fname):
    with open(fname,'r') as f:
        for line in f:
            items = line.split()
            n = len(items)
            
            if 'data points' in line:
                no_of_shells = int(items[n-1])
                df = pd.DataFrame(columns=np.arange(int(items[n-1])),
                         index=pd.Index([],
                                        name='element'),
                         dtype=np.float64)            
            if 'Velocity' in line:
                velocity = extract_block(f)
            if 'Density' in line:
                density = extract_block(f)
            if 'Electron density' in line:
                elec_density = extract_block(f)
            if 'mass fraction\n' in line:
                    abundances = []
                    element_string = items[0]
                    atomic_no = get_index(element_string.capitalize())
                    element_symbol = atomic_dataset.atom_data.loc[atomic_no]['symbol']
                    
                    #Its a Isotope
                    if n==4:
                        element_symbol+=items[1]
                    
                    for line in f:
                        items =line.split()
                        if items:
                            abundances.extend( np.array(items).astype(np.float64))
                        else:
                            break
                    
                    df.loc[element_symbol] = abundances
    print density
    return df.transpose()
                

In [49]:
df = parse_file('DDC15_SN_HYDRO_DATA_0.976d')
#df.to_csv('test.csv',index=False,sep=' ')

[  2.03070640e-18   2.03102960e-18   2.03717390e-18   2.05017290e-18
   2.07641860e-18   2.12992180e-18   2.24110030e-18   2.54418690e-18
   2.98250920e-18   3.60387980e-18   4.49776800e-18   5.77507510e-18
   7.61375770e-18   1.02661100e-17   1.40983690e-17   1.96738090e-17
   2.77635650e-17   3.95697560e-17   5.67580460e-17   8.18252580e-17
   1.18335620e-16   1.71540130e-16   2.49201860e-16   3.62766190e-16
   5.29659080e-16   7.76028600e-16   1.14390060e-15   1.70263150e-15
   2.61147960e-15   4.05966740e-15   6.17606610e-15   8.79369840e-15
   1.19136870e-14   1.81069000e-14   3.41732910e-14   7.38634580e-14
   1.54230530e-13   2.74662680e-13   4.27788900e-13   6.15918470e-13
   8.19270580e-13   1.10932490e-12   1.50430230e-12   1.97092700e-12
   2.52714490e-12   3.27139160e-12   4.44873500e-12   6.26413150e-12
   8.62326020e-12   1.16814180e-11   1.64281650e-11   2.44205070e-11
   3.48509160e-11   4.20977240e-11   5.10960890e-11   6.42290210e-11
   7.27875330e-11   7.31046280e-11

TypeError: Required argument 'object' (pos 1) not found